## Running Ollama

In [1]:
from langchain_community.llms import Ollama

### Loading engineered finance_gpt_llama2 model

In [3]:
llm = Ollama(model="finance_gpt_llama2")
#llm.invoke("What are best practices for financial planning?")

In [12]:
# Read transaction data
import pandas as pd
df = pd.read_csv("/Users/mrinoyb2/git/FinanceGPT/data/transactions/amex_2023.csv")
#df

In [13]:
import pandas as pd
import re

# Assuming df is your DataFrame and "Description" is the column you want to clean

# Define a function to clean and normalize a single description
def clean_normalize_description(description):
    # Convert to lower case
    description = str(description).lower()
    # Replace multiple spaces with a single space
    description = re.sub(r'\s+', ' ', description)
    # Remove special characters
    description = re.sub(r'[^a-z0-9\s]', '', description)
    # Remove numbers
    description = re.sub(r'\d', '', description)
    # Keep only the first 3 words
    description = ' '.join(description.split()[:3])
    # Remove state abbreviations
    description = re.sub(r'\b(?:al|ak|az|ar|ca|co|ct|de|fl|ga|hi|id|il|in|ia|ks|ky|la|me|md|ma|mi|mn|ms|mo|mt|ne|nv|nh|nj|nm|ny|nc|nd|oh|ok|or|pa|ri|sc|sd|tn|tx|ut|vt|va|wa|wv|wi|wy)\b', '', description)
    # Remove hyphens
    description = description.replace('-', ' ')
    # Remove leading and trailing spaces
    description = description.strip()
    return description

# Apply the cleaning function to the "Description" column
df["Description"] = df["Description"].apply(clean_normalize_description)

# Display the DataFrame to verify the cleaned descriptions
df[["Description"]]

# Save the cleaned DataFrame to a new CSV file
df.to_csv("/Users/mrinoyb2/git/FinanceGPT/data/transactions/amex_2023_cleaned.csv", index=False)

# Create a table with Date, Description and Amount
unique_transactions = df[["Date", "Description", "Amount"]]

# Convert dataframe to strings where each column is separated by a '-'
unique_transactions = unique_transactions.astype(str).apply(' - '.join, axis=1)
unique_transactions



0             12/30/2023 - aplpay subway durham - 11.81
1       12/30/2023 - sprouts farmers markdurham - 13.96
2              12/28/2023 - harris teeter durham - 8.85
3         12/27/2023 - moes southwest grildurham - 6.44
4        12/26/2023 - moes southwest grildurham - 11.28
                             ...                       
581    01/07/2023 - aplpay starbucks stochicago - 13.52
582             01/03/2023 - amex dining credit - -10.0
583                01/02/2023 - aplpay starbucks - 10.0
584          01/01/2023 - aplpay subway chicago - 19.91
585                01/01/2023 - taco and burrito - 9.86
Length: 586, dtype: object

In [15]:
# Get index list
def hop(start, stop, step):
    while start < stop:
        yield start
        start += step

index_list = list(hop(0, len(unique_transactions), 10))
index_list

[0,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 210,
 220,
 230,
 240,
 250,
 260,
 270,
 280,
 290,
 300,
 310,
 320,
 330,
 340,
 350,
 360,
 370,
 380,
 390,
 400,
 410,
 420,
 430,
 440,
 450,
 460,
 470,
 480,
 490,
 500,
 510,
 520,
 530,
 540,
 550,
 560,
 570,
 580]

In [34]:
%pip install pydantic

Note: you may need to restart the kernel to use updated packages.


In [17]:
# Output validation
from pydantic import BaseModel, field_validator
from typing import List

# Validate response format - check if it actually contains hyphen ("-")
class ResponseChecks(BaseModel):
    data: List[str]

    @field_validator("data")
    def check(cls, value):
        for item in value:
            if len(item) > 0:
                assert "-" in item, "String does not contain hyphen."

# Test validation
ResponseChecks(data = ['10/10/2023 - Spotify - 10.99 - Entertainment', '10/10/2023 - hi - 5.00 - hi'])

ResponseChecks(data=None)

In [18]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("You are a financial advisor. Can you assign an appropriate category to each transaction. Maintain the format: Date - Description - Amount - Category. Category names should be consistent and less than 3 words.  " + transaction_names)
    response = response.split('\n')

    # Keep only the lines in between blank lines (removing the explaination lines at the beginning and end of the response)
    blank_indexes = [index for index in range(len(response)) if response[index] == '']
    if len(blank_indexes) == 1:
        response = response[(blank_indexes[0] + 1):]
    else:
        response = response[(blank_indexes[0] + 1) : blank_indexes[1]]

    # Print response and validate if it is in the correct format
    print(response)
    ResponseChecks(data = response)
    
    # Put the response in a DataFrame in the format of the input DataFrame (Date, Description, Amount, Category). Separated by " - "
    categories_df = pd.DataFrame(columns=["Date", "Description", "Amount", "Category"])
    for i in range(len(response)):
        categories_df.loc[i] = response[i].split(" - ")
    return categories_df

In [21]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()
max_tries = 5

# Loop through the index_list
for i in range(0, len(index_list) - 1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    # Try and validate output, if it fails, try again for max_tries=5 times
    for j in range(1, max_tries):
        try:
            categories_df = categorize_transactions(transaction_names, llm)
            categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)
            
        except:
            if j < max_tries:
                continue
            else:
                raise Exception(f"Cannot categorize transactions indexes {i} to {i+1}.")
        break

['Date - Description - Amount - Category', '12/30/2023 - Aplpay subway Durham - $11.81 - Transportation', '12/30/2023 - Sprouts Farmers Market Durham - $13.96 - Groceries', '12/28/2023 - Harris Teeter Durham - $8.85 - Groceries', "12/27/2023 - Moe's Southwest Grill Durham - $6.44 - Dining Out", "12/26/2023 - Moe's Southwest Grill Durham - $11.28 - Dining Out", '12/25/2023 - Amc Online Leawood - $6.94 - Entertainment', '12/23/2023 - Total Access Urgent Care - $45.00 - Healthcare', '12/23/2023 - Uber - $18.52 - Transportation', '12/22/2023 - Aplpay Chipotle Durham - $9.03 - Groceries']
['Date - Description - Amount - Category', '12/21/2023 - Walgreens Durham - $13.43 - Personal Care', '12/20/2023 - Kohls.com - $53.74 - Retail', '12/20/2023 - Target Durham - $6.92 - Shopping', "12/19/2023 - Aplpay McDonald's FCary - $2.15 - Dining", '12/18/2023 - Chick-fil-A Cary - $5.73 - Dining', "12/18/2023 - McDonald's FCary - $2.15 - Dining", '12/18/2023 - Target Cary West - $32.45 - Shopping', '12/1

In [22]:
categories_df_all.head()

# print the categories_df_all dataframe. I want to see all rows
pd.set_option('display.max_rows', None)
categories_df_all

,Date,Description,Amount,Category
0,Date,Description,Amount,Category
1,12/30/2023,Aplpay subway Durham,$11.81,Transportation
2,12/30/2023,Sprouts Farmers Market Durham,$13.96,Groceries
3,12/28/2023,Harris Teeter Durham,$8.85,Groceries
4,12/27/2023,Moe's Southwest Grill Durham,$6.44,Dining Out
5,12/26/2023,Moe's Southwest Grill Durham,$11.28,Dining Out
6,12/25/2023,Amc Online Leawood,$6.94,Entertainment
7,12/23/2023,Total Access Urgent Care,$45.00,Healthcare
8,12/23/2023,Uber,$18.52,Transportation
9,12/22/2023,Aplpay Chipotle Durham,$9.03,Groceries


In [23]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array(['Category', 'Transportation', 'Groceries', 'Dining Out',
       'Entertainment', 'Healthcare', 'Personal Care', 'Retail',
       'Shopping', 'Dining', 'Grocery', 'Utilities', 'Miscellaneous',
       'Beverages', 'Food', 'Food Delivery', 'Online Shopping',
       'Clothing and Accessories', 'Food & Beverage',
       'Software & Services', 'Alcohol', 'Travel', 'Income',
       'Food and Dining', 'Subscription',
       'Miscellaneous (negative amount indicates a refund)',
       'Dining Credit', 'Moving Expenses', 'Business Expenses',
       'Online Learning', 'Travel Expenses', 'Personal Expenses',
       'Credit Card Rewards', 'Food & Dining', 'Personal Income',
       'Food and Beverage', 'Moving & Storage', 'Professional Services',
       'Personal Services', 'Communications', 'Online Services',
       'Rent/Mortgage', 'Insurance', 'Education', 'Housing',
       'Accommodation', 'Credits/Refunds', 'Miscellaneous (Loss)',
       'Miscellaneous (loss)', 'Daily Expenses', 'Online 

In [29]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Retail/Dining"), 'Category'] = "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Retail/Fast"), 'Category'] = "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Entertainment (Food)"), 'Category'] = "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Dining"), 'Category'] = "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Cuisine"), 'Category'] = "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Restaurant"), 'Category'] = "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Asian"), 'Category'] = "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Mexican"), 'Category'] = "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Beverage"), 'Category'] = "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Eat"), 'Category'] = "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Drink"), 'Category'] = "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Coffee"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Wellness"), 'Category'] = "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health"), 'Category'] = "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Fitness"), 'Category'] = "Health and Wellness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Accommodation"), 'Category'] = "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Airline"), 'Category'] = "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Airplane"), 'Category'] = "Travel"

categories_df_all.loc[categories_df_all['Category'].str.contains("Transport"), 'Category'] = "Transportation"

categories_df_all.loc[categories_df_all['Category'].str.contains("Housing"), 'Category'] = "Housing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Rent"), 'Category'] = "Housing"

categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing and Accessories"

categories_df_all.loc[categories_df_all['Category'].str.contains("Grocer"), 'Category'] = "Groceries"

categories_df_all.loc[categories_df_all['Category'].str.contains("Recreation"), 'Category'] = "Entertainment"
categories_df_all.loc[categories_df_all['Category'].str.contains("Entertainment"), 'Category'] = "Entertainment"

categories_df_all.loc[categories_df_all['Category'].str.contains("Tele"), 'Category'] = "Communications"
categories_df_all.loc[categories_df_all['Category'].str.contains("communication"), 'Category'] = "Communications"

categories_df_all.loc[categories_df_all['Category'].str.contains("Online Payment"), 'Category'] = "Miscellaneous"
categories_df_all.loc[categories_df_all['Category'].str.contains("Online"), 'Category'] = "Miscellaneous"
categories_df_all.loc[categories_df_all['Category'].str.contains("Retail"), 'Category'] = "Miscellaneous"
categories_df_all.loc[categories_df_all['Category'].str.contains("Shopping"), 'Category'] = "Miscellaneous"
categories_df_all.loc[categories_df_all['Category'].str.contains("Online Payment"), 'Category'] = "Miscellaneous"
categories_df_all.loc[categories_df_all['Category'].str.contains("Online Payment"), 'Category'] = "Miscellaneous"
categories_df_all.loc[categories_df_all['Category'].str.contains("Miscellaneous"), 'Category'] = "Miscellaneous"
categories_df_all.loc[categories_df_all['Category'].str.contains("Category"), 'Category'] = "Miscellaneous"

/var/folders/14/bb_z175163b7dlwgrfsth0k40000gn/T/ipykernel_10230/1533456042.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  categories_df_all.loc[categories_df_all['Category'].str.contains("Entertainment (Food)"), 'Category'] = "Food and Drinks"


In [30]:
# Print unique categories
unique_categories = categories_df_all["Category"].unique()
unique_categories

array(['Miscellaneous', 'Transportation', 'Groceries', 'Food and Drinks',
       'Entertainment', 'Health and Wellness', 'Personal Care',
       'Utilities', 'Clothing and Accessories', 'Services', 'Alcohol',
       'Travel', 'Income', 'Subscription', 'Moving Expenses',
       'Business Expenses', 'Personal Expenses', 'Credit Card Rewards',
       'Personal Income', 'Moving & Storage', 'Communications', 'Housing',
       'Insurance', 'Education', 'Credits/Refunds', 'Daily Expenses',
       'Software Subscription', 'Education Expenses', 'Snacks', 'Gas',
       'Expense', 'Credit Card Fee', 'Gym Membership', 'Investment Loss',
       'Investment Gain', 'Communication'], dtype=object)

In [33]:
categories_df_all.to_csv("/Users/mrinoyb2/git/FinanceGPT/data/transactions/amex_transactions_2023_categorized.csv", index=False)

In [ ]:
# Aggregate transactions 
